# 작업형2
- 빅데이터 분석기사 2회 실기
- 데이터 다운로드 : https://www.kaggle.com/prachi13/customer-analytics
- 데이터 파일 명 : Train.csv (파일명이 다를 경우 파일명을 수정해주세요)
- (데이터가 삭제될 시 수업노트에 유사 데이터 셋을 올려두겠습니다)


## 제품 배송 시간에 맞춰 배송되었는지 예측모델 만들기
- 제공된 데이터
    - X_train.csv
    - y_train.csv
    - X_test.csv

- 학습용 데이터 (X_train, y_train)을 이용하여 배송 예측 모형을 만든 후, 이를 평가용 데이터(X_test)에 적용하여 얻은 예측(시간에 맞춰 도착하지 않을 확률)값을 다음과 같은 형식의 CSV파일로 생성하시오(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)
- 0 정시 도착, 1 정시 도착하지 않음

```
ID,Reached.on.Time_Y.N
3500,0.267
3501,0.578
3502,0.123

```

- 출력을 원하실 경우 print() 함수 활용
- 예시) print(df.head())
- getcwd(), chdir() 등 작업 폴더 설정 불필요
- 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

- 데이터 파일 읽기 예제

```
# 데이터 불러오기
import pandas as pd
X_test = pd.read_csv("data/X_test.csv")
X_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv")
```

- csv파일 생성 예시

```
pd.DataFrame({'ID': X_test['cust_id'], 'Reached.on.Time_Y.N': pred}).to_csv('003000000.csv', index=False)
```


In [1]:
# 시험환경 세팅 3개 파일 생성 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name

    if null_name != "":
        df[df == null_name] = np.nan

    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2022)
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test

df = pd.read_csv("Train.csv") # 파일명이 다를 경우 파일명을 수정해주세요
X_train, X_test, y_train, y_test = exam_data_load(df, target='Reached.on.Time_Y.N', id_name='ID')
X_train.to_csv("X_train.csv", index=False)
y_train.to_csv("y_train.csv", index=False)
X_test.to_csv("X_test.csv", index=False)

In [12]:
# 데이터 불러오기
import pandas as pd
X_test = pd.read_csv("X_test.csv")
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")

X_train.shape, y_train.shape, X_test.shape

((8799, 11), (8799, 2), (2200, 11))

In [34]:
# 데이터 불러오기
import pandas as pd
X_test = pd.read_csv("X_test.csv")
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")

X_train.shape, y_train.shape, X_test.shape
# EDA
# print(X_train.info(), y_train.info())  # null 값 없음. 인코딩 필요('Warehouse_block', 'Mode_of_Shipment', 'Product_importance'), ID 삭제
# print(X_train.describe()) # ['Customer_care_calls', 'Customer_rating', 'Prior_purchases'] #=> 숫자형이지만 범주형

# 전처리(기타)
X_train = X_train.drop(['ID', 'Gender'], axis = 1)
X_test = X_test.drop(['ID', 'Gender'], axis = 1)
y_train = y_train.drop(['ID'], axis = 1)

cols_object = list(X_train.select_dtypes(include='object').columns)
cols_numeric = list(X_train.select_dtypes(exclude='object').columns)

cols_not_scale = cols_object + ['Customer_care_calls', 'Customer_rating', 'Prior_purchases']

# 전처리(인코딩)
from sklearn.preprocessing import LabelEncoder

for col in cols_object:
  le = LabelEncoder()
  X_train[col] = le.fit_transform(X_train[col])
  X_test[col] = le.transform(X_test[col])

# 전처리(스케일링)

# 검증 데이터 분리
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.15, random_state = 2022)

# 모델링
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state = 2022, max_depth = 3, n_estimators = 200)
clf.fit(X_tr, y_tr)
pred = clf.predict_proba(X_val)

# 평가
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_val, pred[:, 1]))

# 제출
pred = clf.predict_proba(X_test)
submit = pd.DataFrame({
    'Reached.on.Time_Y.N' : pred[:, 1]
}).to_csv('adsf.csv', index=False)

<ipython-input-34-e6e49883587c>:40: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_tr, y_tr)


0.7602058168101962
